# ***Recommendation Algo***



In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# # pip install matplotlib


In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
df = pd.read_csv(Path("data.csv"))
df

,name,released_date,tags,url,description,developers,publishers,price,all_reviews,positice_reviews,...,Role-Playing,Management & Building,Stealth & Strategy,Exploration & Adventure,Fantasy & Mythology,Education & Learning,VR,Other,price_range,score_range
0,Defend Earth: Xenos Survivors,2023-01-12,"['Roguelite', 'Bullet Hell', 'Action Roguelike...",https://store.steampowered.com/app/2155280,Defend yourself against an onslaught of monstr...,Reija,Reija,33,20,17,...,0,0,1,0,0,0,0,0,0-100,70-90
1,The Chaput's Baby,2022-09-26,"['Horror', 'Supernatural', 'Mystery', 'Demons'...",https://store.steampowered.com/app/2155300,The Chaput church was a well know place by the...,Gundes,Gundes,20,24,21,...,1,0,0,1,0,0,0,0,0-100,70-90
2,风色旅人,2022-11-17,"['Adventure', 'Action', 'Strategy', 'RPG', 'Ac...",https://store.steampowered.com/app/2155310,《风色旅人》是一款3D异世界冒险游戏，用开放式的场景、精美的人物建模、炫酷的技能搭造了一场奇...,风色旅人,风色旅人,0,558,428,...,0,0,1,1,0,0,0,0,0-100,70-90
3,勇者联盟星耀传说,2022-09-29,"['Adventure', 'RPG', 'Turn-Based Strategy', 'J...",https://store.steampowered.com/app/2155390,There are forty characters that can join the p...,心之初,心之初,38,20,18,...,0,0,1,1,1,0,0,0,0-100,90-99
4,Usagi Health Club,2022-09-19,"['Casual', 'Visual Novel', 'Choose Your Own Ad...",https://store.steampowered.com/app/2155420,A casual visual novel with minigames that let ...,PUSH! Soft,Push Publication,95,5,0,...,1,0,0,1,1,0,0,0,0-100,0-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38959,Factory Town Idle,2023-04-20,"['Casual', 'Idler', 'Minimalist', '2D', 'Click...",https://store.steampowered.com/app/2207490,Factory Town: Idle is an incremental-style Tow...,Erik Asmussen,Erik Asmussen,33,141,121,...,0,1,0,1,1,0,0,0,0-100,70-90
38960,EA SPORTS™ PGA TOUR™,2023-04-06,"['RPG', 'Sports', 'Co-op', 'Local Co-Op', 'Loc...",https://store.steampowered.com/app/1677350,"The exclusive home of the Majors, EA SPORTS™ P...",Electronic Arts - Tiburon,Electronic Arts,109,625,332,...,0,0,0,0,0,0,0,0,100-150,50-70
38961,Star Realms,2016-03-08,"['Deckbuilding', 'Strategy', 'Card Battler', '...",https://store.steampowered.com/app/438140,Star Realms is a hugely popular space combat d...,"Wise Wizard Games, LLC","Wise Wizard Games, LLC",0,1720,1485,...,0,1,1,0,0,0,0,0,0-100,70-90
38962,World of Tanks,2021-04-28,"['Tanks', 'Free to Play', 'Action', 'Vehicular...",https://store.steampowered.com/app/1407200,Jump into the free-to-play team-based shooter ...,Wargaming Group Limited,Wargaming Group Limited,0,65620,56520,...,0,0,1,0,0,0,0,0,0-100,70-90


# **Recommendation Model**



In [3]:
# 删除不需要的列
df= df.drop('released_date', axis=1)
df = df.drop('tags', axis=1)
df = df.drop('url', axis=1)
# df = df.drop('description', axis=1)
df = df.drop('Other', axis=1)
df = df.drop('price_range', axis=1)
df = df.drop('score_range', axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38964 entries, 0 to 38963
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     38964 non-null  object 
 1   description              38964 non-null  object 
 2   developers               38964 non-null  object 
 3   publishers               38964 non-null  object 
 4   price                    38964 non-null  int64  
 5   all_reviews              38964 non-null  int64  
 6   positice_reviews         38964 non-null  int64  
 7   review_score             38964 non-null  float64
 8   supported_languages      38964 non-null  object 
 9   released_year            38964 non-null  int64  
 10  Action                   38964 non-null  int64  
 11  First-Person             38964 non-null  int64  
 12  Third Person             38964 non-null  int64  
 13  Adventure                38964 non-null  int64  
 14  Strategy              

In [5]:

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, LabelEncoder
#  first select the features that need to be used to calculate the similarity
list_of_tags = [ 'Action','First-Person','Third Person', 'Adventure', 'Strategy', 'RPG', 'Simulation', 'Horror', 'Survival', 'Puzzle',
  'Indie',  'Retro',  'Pixel Graphics',  'Rouguelike',  '2D Platform', '3D Platform',  'Multiplayer',  'Casual',  'Visual Novel',  'Otome',  'Arcade',  'Racing',  'Sports',  'Early Access',  'Mature Content',
  'Atmospheric',  'Narrative',  'Sci-Fi & Fantasy',
  'Supernatural',  'Post-apocalyptic',  'Humor',  'Music & Sound',
  'Art & Design',  'Role-Playing',  'Management & Building',  'Stealth & Strategy',  'Exploration & Adventure', 'Fantasy & Mythology', 'Education & Learning','VR']



In [6]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack, csr_matrix

# First select the features that need to be used to calculate the similarity
features = ['developers', 'publishers', 'price', 'review_score', 'supported_languages', 'released_year'] + list_of_tags
df_features = df[features]

# For numerical features, we need to normalize
num_features = ['price', 'review_score', 'released_year']
scaler = StandardScaler()
df_features[num_features] = scaler.fit_transform(df_features[num_features])

# For categorical features, we do label encoding
cat_features = ['developers', 'publishers', 'supported_languages']
for feature in cat_features:
    le = LabelEncoder()
    df_features[feature] = le.fit_transform(df_features[feature])

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Use the description attribute to fit the vectorizer, and convert the description to a vector
tfidf_matrix = vectorizer.fit_transform(df['description'])

# Convert the original feature matrix to a sparse matrix in CSR format
features_sparse = csr_matrix(df_features.values)

# Use the hstack function to stack the TF-IDF matrix and feature matrix horizontally
combined_features = hstack([features_sparse, tfidf_matrix])

# Calculate cosine similarity
cosine_sim = cosine_similarity(combined_features, combined_features)

# Assume that the index of the game the user already likes is idx
# We can find the 10 games most similar to this game
idx = 0
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:11]

# Get the index of similar games
game_indices = [i[0] for i in sim_scores]

# Output similar games
print(df['name'].iloc[game_indices])


/var/folders/mv/5yb01s9n2fl4vmyfkpmjp85r0000gn/T/ipykernel_2518/3581431224.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features[num_features] = scaler.fit_transform(df_features[num_features])
/var/folders/mv/5yb01s9n2fl4vmyfkpmjp85r0000gn/T/ipykernel_2518/3581431224.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features[feature] = le.fit_transform(df_features[feature])
/var/folders/mv/5yb01s9n2fl4vmyfkpmjp85r0000gn/T/ipykernel_2518/3581431224.py:19: SettingWithCopyWarning: 
A value is 

21606               Zombardment
8146     Manual Intervention VR
12027             Charge Champs
19425             Cramit's Keep
7242                Tor Eternum
12425               Tor Eternum
9283                    ShapeVS
5365             Invader Signal
2095             Decks Of Power
34603                Pixel Star
Name: name, dtype: object


In [10]:

game_indices = pd.Series(df.index, index=df['name']).drop_duplicates()

def get_recommendations(game_name, cosine_sim=cosine_sim):
    idx = game_indices[game_name]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    similar_game_indices = [i[0] for i in sim_scores]

    return df['name'].iloc[similar_game_indices]

# Get recommended games
recommended_games = get_recommendations('Charge Champs')
print(recommended_games)


14953                    Burst Drive
26887                  TUNNEL DIVERS
9283                         ShapeVS
17293                       Red Blue
1020                Super Jumpy Ball
21606                    Zombardment
25418                     Iron Snout
4717     Chickenoidz Super Pre-Party
24275                    Toaster Jam
34603                     Pixel Star
Name: name, dtype: object
